# Text Analysis across Multiple Texts

## Part I: Importing Python Packages or Libraries

1. Before beginning, we need to import some packages. Often, we need to install and import customized Python packages (sometimes called "modules") in addition to the core functions (like **print()**, **len()**, **sum()**, and others).

In [ ]:
import os, pathlib, glob #these packages allow us to navigate through the files on our own computers
import collections   #allows us to create frequency counts
from pathlib import Path #the pathlib package helps us work with file paths
#for more on using pathlib see: https://builtin.com/software-engineering-perspectives/python-pathlib
import pandas as pd #for creating and working with dataframes
import nltk, re #we can import multiple packages on one line using commas to separate new package names
from nltk import RegexpTokenizer 
from nltk.corpus import stopwords
from nltk import ngrams, pos_tag, word_tokenize, sent_tokenize
nltk.download('punkt')
#import matplotlib as plt   #matplotlib and seaborn are used here to create graphs, charts, and other visualizations
import matplotlib.pyplot as plt #needed for xticks
import seaborn as sns

#code below modifies how plots will be shown in this notebook
plt.rcParams['figure.figsize'] = [12, 8]  #changes default figure size to make larger plots
%config InteractiveShellApp.matplotlib = 'inline'
%config InlineBackend.figure_formats = ['svg']

#Press CTRL+Enter to run this codeblock! 

2. Let's take a look at the contents of our corpus stored in the "state-of-the-union-dataset/txt" directory.

In [ ]:
#sotudir = Path(Path.cwd().parent.parent, "state-of-the-union-dataset","txt")
sotudir = Path("state-of-the-union-dataset","txt")
pathlist = sorted(sotudir.glob('*.txt'))      # .glob only stores the pathlist temporarily (for some reason), so you need to call it again!2
for path in pathlist:
    print(path)

## II. Create a Dataframe of SOTU texts

**3. DATAFRAMES:** To enable easier analysis of the SOTU texts, we can store info about each in a **dataframe**. A dataframe in Python is a common data structure enabled with the **pandas** library. It is a two-dimensional data table that stores data in rows and columns. 

Run the code below, and then examine what each portion of the code does.

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
#n=50

txtList=[]
pathlist = sorted(sotudir.glob('*.txt'))      # .glob only stores the pathlist temporarily (for some reason), so you need to call it again!2
for path in pathlist:
    fn=path.stem                       #stem returns the filename minus the ".txt" (file extension). 
    year,pres=fn.split("_")            # fn = "1794_Washington" becomes year = "1794" and pres = "Washington"
    with open(path,'r') as f:  
        sotu = f.read()                #opens each file and reads it in as "sotu"
    tokens=tokenizer.tokenize(sotu)    # tokenizes "sotu"
    numtoks = len(tokens)             # counts the number of tokens in "sotu"
    txtList.append([pres,year,numtoks,tokens,sotu])   #add this info for "sotu" to a running list for all sotu addresses

txtList            

4. Now we want to read in this list of information into a dataframe.

In [ ]:
colnames = ['pres', 'year', 'numtoks', 'tokens', 'fulltext']
sotudf = pd.DataFrame(txtList, columns = colnames)  #places our completed list of SOTU info in a dataframe
sotudf.head(10)                                #prints out the first 10 rows of this dataframe (the default value for head() is 5 rows)

4b. Let's next sort this dataframe by year. We will also want to reset the index to reflect this re-sorting.

In [ ]:
sotudf = sotudf.sort_values(by = "year")
sotudf = sotudf.reset_index(drop = True)
sotudf.head(10)

<div style = "background-color:#f3e5f5">
<h3 style = "color:purple">III. Python Basics: Writing Functions</h3>

<p style="color:purple"><b>5. FUNCTIONS:</b> Next, we would like to create some new columns with lists of tokens that are lower-cased and with stopwords removed. To do so, it is helpful to write a function that does this for a single text. Then, we can apply that function across the entire corpus of SOTU addresses stored in this dataframe.</p>

<p style="color:purple">We have already used a variety of core Python functions such as <b>sum()</b>, <b>len()</b>, and <b>print()</b>. We have also called on many functions defined in auxiliary Python libraries or packages: such as the <b>word_tokenize()</b> and <b>concordance</b> functions from the <b>nltk</b> library we imported.</p>

<p style="color:purple">Here, however, we will create our own function. The typical format of a Python function is:</p>

```python
def functionName(argumentsToPassIn):
    function instructions
    return(resultsOfFunction)
```
    
<p style="color:purple"><b>6. A SIMPLE FUNCTION:</b>So, for example, if we had a list of names and we wanted to create a function to retrieve the initial of each, we could use the following function:</p>
</div>

In [ ]:
def Initials(fullname):
    '''
    this function reads in a person's full name in this format "John Doe" or "Jane Q. Doe"
    and returns the initials
    '''
    caps = re.findall('([A-Z])', fullname) #this use sthe findall function from the re package to find all capitalized letters
    inits = ''.join(caps)  #takes our list of capitalized letters stored in "caps" and concatenates it
    return(inits)
    
fullname = "Jeremy M. Mikecz"     #replace w/ your name
Initials(fullname)

<div style = "background-color:#f3e5f5"><p style="color:purple">7. We can now apply this function to quickly return the initials from a long list of names.</p></div>

In [ ]:
actorlist = ['Christoph Waltz','Tom Hardy','Doug Walker','Daryl Sabara','J.K. Simmons','Brad Garrett','Chris Hemsworth','Alan Rickman','Henry Cavill','Kevin Spacey','Giancarlo Giannini','Johnny Depp','Johnny Depp','Henry Cavill','Peter Dinklage','Chris Hemsworth','Johnny Depp','Will Smith','Aidan Turner','Emma Stone','Mark Addy','Aidan Turner','Christopher Lee','Naomi Watts','Leonardo DiCaprio','Robert Downey Jr.','Liam Neeson','Bryce Dallas Howard','Albert Finney','J.K. Simmons','Robert Downey Jr.','Johnny Depp','Hugh Jackman','Steve Buscemi','Glenn Morshower','Bingbing Li','Tim Holmes','Emma Stone','Jeff Bridges','Joe Mantegna','Ryan Reynolds','Tom Hanks','Christian Bale','Jason Statham','Peter Capaldi','Jennifer Lawrence','Benedict Cumberbatch','Eddie Marsan','Leonardo DiCaprio','Jake Gyllenhaal','Charlie Hunnam','Glenn Morshower','Harrison Ford','A.J. Buckley','Kelly Macdonald','Sofia Boutella','John Ratzenberger','Tzi Ma','Oliver Platt','Robin Wright','Channing Tatum','Christoph Waltz','Jim Broadbent','Jennifer Lawrence','Christian Bale','John Ratzenberger','Amy Poehler','Robert Downey Jr.','ChloÃ« Grace Moretz','Will Smith','Jet Li','Will Smith','Jimmy Bennett','Tom Cruise','Jeanne Tripplehorn','Joseph Gordon-Levitt','Amy Poehler','Scarlett Johansson','Robert Downey Jr.','Chris Hemsworth','Angelina Jolie Pitt','Gary Oldman','Tamsin Egerton','Keanu Reeves','Scarlett Johansson','Jon Hamm','Judy Greer','Damon Wayans Jr.','Jack McBrayer','Tom Hanks','Vivica A. Fox','Gerard Butler','Nick Stahl','Bradley Cooper','Matthew McConaughey','Leonardo DiCaprio','Mark Chinnery','Aidan Turner','Paul Walker','Brad Pitt','Jennifer Lawrence','Jennifer Lawrence','Nicolas Cage','Jimmy Bennett','Johnny Depp','Justin Timberlake','Dominic Cooper','J.K. Simmons','Bruce Spence','Jennifer Garner','Zack Ward','Anthony Hopkins','Robert Pattinson','Robert Pattinson','Will Smith','Will Smith','Johnny Depp','Janeane Garofalo','Christian Bale','Bernie Mac','Robin Williams','Hugh Jackman','Essie Davis','Josh Gad','Steve Bastoni','Chris Hemsworth','Tom Hardy','Tom Hanks','Chris Hemsworth','ChloÃ« Grace Moretz','Kelli Garner','Liam Neeson','Johnny Depp','Tom Cruise','Anthony Hopkins','Christoph Waltz','Matthew Broderick','Angelina Jolie Pitt','Seychelle Gabriel','Philip Seymour Hoffman','Channing Tatum','Elisabeth Harnois','Hugh Jackman','Hugh Jackman','Ty Burrell','Brad Pitt','Jada Pinkett Smith','Toby Stephens','Ed Begley Jr.','Bruce Willis','Will Smith','Robin Wright','J.K. Simmons','Tom Cruise','Hugh Jackman','John Michael Higgins','Tom Cruise','Christian Bale','Chris Hemsworth','J.K. Simmons','Gerard Butler','Gerard Butler','Sam Shepard','Matt Frewer','Jet Li','Kevin Rankin','Channing Tatum','Matthew McConaughey','Steve Buscemi','Chris Evans','Colin Salmon','James DArcy','Robert Pattinson','Robin Williams','Ty Burrell','Don Johnson','Mark Rylance','Leonardo DiCaprio','Ryan Reynolds','Johnny Depp','Benedict Cumberbatch','Matt Damon','Angelina Jolie Pitt','Judy Greer','Jennifer Lawrence','Robert Pattinson','Jim Parsons','Tom Cruise','Will Smith','Salma Hayek','Angelina Jolie Pitt','Anthony Hopkins','Toby Jones','Daniel Radcliffe','Essie Davis','Will Smith','Alfre Woodard','Rupert Grint','Robin Williams','J.K. Simmons','Daniel Radcliffe','Ryan Reynolds','Mark Chinnery','Johnny Depp','Rupert Grint','Jennifer Lawrence','Tom Hanks','Miguel Ferrer','Hugh Jackman','Paul Walker','Robert Downey Jr.','Liam Neeson','Ronny Cox','Tony Curran','Jeremy Renner','Michael Gough','Clint Howard','Jake Gyllenhaal','Tom Cruise','Karen Allen','Chris Evans','Suraj Sharma','Nicolas Cage','Matt Damon','Demi Moore','Michael Fassbender','Nathan Lane','Matt Damon','Vin Diesel','Gary Oldman','Scott Porter','Shelley Conn','Tom Cruise','Morgan Freeman','Natalie Portman','Natalie Portman','Steve Buscemi','Hugh Jackman','Natalie Portman','Ryan Reynolds','Alain Delon','Nicolas Cage','Chris Hemsworth','Noel Fisher','Phaldut Sharma','Jamie RenÃ©e Smith','Stephen Amell','Tim Blake Nelson','Robin Williams','Dwayne Johnson','Vincent Schiavelli','Heath Ledger','Brad Pitt','Brad Pitt','Kate Winslet','Leonardo DiCaprio','James Corden','Christoph Waltz','George Peppard','Eva Green','Mahadeo Shivraj','Steve Buscemi','Naomi Watts','Hugh Jackman','Jacob Tremblay','Jason Patric','Harrison Ford','Bruce Willis','Christopher Lee','Jim Broadbent','Will Smith','Sean Hayes','Will Smith','Liam Neeson','Chazz Palminteri','Oprah Winfrey','Matt Damon','Mathew Buck','Scarlett Johansson','Del Zamora','Nicolas Cage','Djimon Hounsou','Tom Cruise','Daniel Radcliffe','Eva Green','Cary-Hiroyuki Tagawa','Joe Morton','Johnny Depp','Denzel Washington','Jamie Lee Curtis','Denzel Washington','Robert De Niro','Dwayne Johnson','Vanessa Williams','Leonardo DiCaprio','Demi Moore','Eartha Kitt','Jason Statham','Nicolas Cage','Djimon Hounsou','Catherine OHara','Hugh Jackman','Josh Hutcherson','Johnny Depp','CCH Pounder','Leonardo DiCaprio','Leonardo DiCaprio','Michael Gough','Jake Busey','Tom Hanks','Abbie Cornish','Frances Conroy','Dwayne Johnson','Joseph Gordon-Levitt','Will Ferrell','Jason Statham','Ray Winstone','Jamie Kennedy','Chris Hemsworth','Rosario Dawson','Matt Damon','Francesca Capaldi','Ben Gazzara','Dwayne Johnson','Leonardo DiCaprio','Christian Bale','Jeff Bridges','Jon Lovitz','Ioan Gruffudd','Will Ferrell','Milla Jovovich','Chris Noth','Frank Welker','Peter Dinklage','Hayley Atwell','Michael Imperioli','Alexander Gould','Orlando Bloom','Christopher Lee','Jeff Bridges','Angelina Jolie Pitt','Johnny Depp','Michael Jeter','James Franco','Martin Short','Bruce Willis','Dennis Quaid','Holly Hunter','Christopher Masterson','Logan Lerman','Will Smith','Tom Hanks','Denzel Washington','Mei MelanÃ§on','Harrison Ford','Will Forte','Denis Leary','Adam Scott','Bill Murray','Leonardo DiCaprio','Ming-Na Wen','Robert Downey Jr.','Robin Wright','Bruce Willis','Robert Downey Jr.','Morgan Freeman','Leonard Nimoy','Bella Thorne','Tom Cruise','Adam Sandler','Peter Dinklage','Haley Joel Osment','Marsha Thomason','Matthew McConaughey','Greg Grunberg','Curtiss Cook','Logan Lerman','Gerard Butler','Daniel Radcliffe','Alun Armstrong','Brad Pitt','Don Cheadle','Anne Hathaway','Robin Williams','Don Cheadle','Harrison Ford','Liam Neeson','Tim Blake Nelson','William Smith','Paddy Considine','Shirley Henderson','Jeff Bridges','Philip Seymour Hoffman','Paul Walker','Tom Hanks','Robin Williams','Matt Damon','Harrison Ford','Brad Pitt','Milla Jovovich','Steve Buscemi','Jeff Bennett','Caroline Dhavernas','Denzel Washington','Ioan Gruffudd','Matthew Broderick','Kate Winslet','Will Smith','Meryl Streep','Al Pacino','Jon Favreau','Kate Winslet','Bob Hoskins','Dwayne Johnson','F. Murray Abraham','Li Gong','Amber Stevens West','Jim Broadbent','Anthony Hopkins','Raymond Cruz','Roy Scheider','Julia Roberts','Anna Kendrick','Glenn Morshower','Larry Miller','Sarah Michelle Gellar','Wood Harris','Adam Sandler','Ted Danson','Jack McBrayer','Kristen Stewart','Seth MacFarlane','Robert Downey Jr.','Robert Duvall','Morgan Freeman','Jason Statham','Tom Cruise','Jennifer Lawrence','Bradley Cooper','Michael Gough','Bruce Willis','Tia Carrere','Steve Buscemi','Morgan Freeman','Bruce Willis','Adam Sandler','Amy Poehler','Steve Buscemi','Bill Murray','Keanu Reeves','Leonardo DiCaprio','Jon Favreau','Jim Broadbent','Nicolas Cage','Adam Sandler','Tom Hanks','Adam Sandler','Elden Henson','Steve Buscemi','Rosario Dawson','Philip Seymour Hoffman','Denzel Washington','Robin Williams','Liam Neeson','Bill Murray','Roger Rees','Keanu Reeves','Julia Roberts','Brad Pitt','Harrison Ford','Justin Timberlake','Matt Damon','Rosario Dawson','Gary Oldman','Denzel Washington','Vanessa Redgrave','Steve Buscemi','Elizabeth Montgomery','Quincy Jones','Mark Addy','Charlize Theron','Hugh Jackman','Michael Emerson','Robin Williams','Adam Sandler','Matt Damon','Natalie Portman','Nissim Renard','Anthony Hopkins','Bruce Willis','Bruce Greenwood','Sylvester Stallone','Charlie Rowe','Richard Tyson','Brendan Fraser','Fergie','Paul Walker','Olivia Williams','Adam Goldberg','Vin Diesel','Bob Neill','Mia Farrow','Pedro ArmendÃ¡riz Jr.','David Oyelowo','Sasha Roiz','Sariann Monaco','Adam Goldberg','Matthew Broderick','Josh Hutcherson','Will Forte','Philip Seymour Hoffman','J.K. Simmons','Al Pacino','Paul Walker','Jeff Bridges','Roger Rees','Robert De Niro','Steve Coogan','Jason Flemyng','Steve Carell','Will Smith','Ariana Richards','Jada Pinkett Smith','Charlie Hunnam','Hugh Jackman','Angelina Jolie Pitt','Nicolas Cage','Denis Leary','Adam Sandler','Jerry Stiller','James DArcy','Matthew Broderick','Morgan Freeman','Steve Buscemi','Tom Hanks','Harold Perrineau','Don Cheadle','Nicholas Lea','Philip Seymour Hoffman','Robert De Niro','Loretta Devine','Adam Arkin','Dwayne Johnson','Ayelet Zurer','Bruce Willis','Tom Selleck','Henry Cavill','Adam Sandler','Steve Buscemi','Bruce Willis','Julia Ormond','Bai Ling','Henry Cavill','Jimmy Bennett','Matt Damon','Harrison Ford','Connie Nielsen','Christopher Meloni','Brendan Fraser','Dennis Quaid','Robin Wright','Steve Carell','Jon Hamm','Nicolas Cage','Peter Coyote','Peter Dinklage','Matthew McConaughey','Adam Sandler','Jennifer Garner','Will Ferrell','Raven-SymonÃ©','Mhairi Calvey','Jake Gyllenhaal','Albert Brooks','Martin Landau','Sylvester Stallone','David Gant','Bryce Dallas Howard','Oliver Platt','Rory Culkin','Rupert Everett','John Ratzenberger','Julia Roberts','Vin Diesel','Tim Conway','Lili Taylor','Michael Fassbender','Robin Williams','Dwayne Johnson','Bruce Willis','Jeremy Renner','Nicole Beharie','Tom Cruise','Bryce Dallas Howard','Sanaa Lathan','Amy Poehler','Jon Hamm']

In [ ]:
ctr=0
for actor in actorlist:
    if ctr<20:         #we add this conditional so that it only prints out the first 20 examples to save space below
        print(Initials(actor),"=",actor)
        ctr+=1

##without the if command: 
#for actor in actorlist:
#    print(Initials(actor),"=",actor)


## IV. Apply a function across an entire column to create a new column

8. As in the previous notebook, we can lower-case our token list and then remove all stopwords.

However, since all texts in our corpus are now stored in a dataframe, we can use the **.apply()** method to do this all at once across the entire dataframe.

The .apply() method follows this format:

```
[new_column] = [column].apply(function to apply across entire column)
```

There are two main ways to apply a function in this way:

1. We can define an external function and then call it:

```
def some_function(x):  #x is the value stored in the column
    [instructions]
[new_column] = [column].apply(some_function)

#or when you have multiple arguments for your function:

def some_function(x, y):  #x is the value stored in the column, y is an additional parameter
    [instructions]

[new_column] = [column].apply(some_function, y)
```

2. We can create a short, "anonymous" function using **lambda**:

```
[new_column] = [column].apply(lambda x: [what to do with x])  #x is the value stored in the column
```

In the following code cell, we will use the second technique to create a new column with lower-case tokens derived from the tokens column


In [ ]:
sotudf['ltoks'] = sotudf['tokens'].apply(lambda x: [tok.lower() for tok in x]) #creates a new column to store lower case tokens
sotudf.head(10) 

9. What do you think the following code does?

In [ ]:
stop = stopwords.words('english')
sotudf['ltoks_ns'] = sotudf['ltoks'].apply(lambda x: [ltok for ltok in x if ltok not in stop])
sotudf.head(10) 

**10. SAVING DATAFRAMES:** It would be useful to reuse this dataframe in the future. Let's save it to a csv file. 

In [ ]:
sotudf.to_csv("sotudf.tsv", encoding = "utf-8",sep = "\t")
#we are saving this with a "tsv" extension to indicate we are using tabs ("\t") as our delimiter between columns, not commas
#csv = comma separated values; tsv = tab separated values
#within JupyterHub you should see this new file appear on the folder directory to the left. 
#you are welcome to download it (right click--> Download on PCS; Ctrl-Click --> Download on Macs)
# you can then open it in Excel by 
# 1. opening a new, blank Excel workbook; 
# 2. Go to the Data tab --> GetData/From Text/CSV
# 3. navigate to the folder you downloaded the tsv file to; 
# 4. change the option at the bottom right to view "All Files" 
# 5. select and open the .tsv file 6. in the dialogue box make sure the delimiter is set to "Tab" (and also recommended but not required to set File Origin to "Unicode (UTF-8)" 
# 6. Select Load

## V. Calculating frequencies of an individual word

Now, we are going to search across the entire SOTU corpus for a particular search term, count its frequency in each text using a function we create, and then store that count in a dataframe.

11. First, we will apply a function to search for a term within a specific tokenized list. 

In [ ]:
def getWordFreq (ltoks, term):
    #ltoks = [tok.lower() for tok in toks]
    tokfreqs=collections.Counter(ltoks)
    wordFreq = tokfreqs[term]
    return(wordFreq)

11b. Let's test that function on one text: with the words stored in our ltokens2 list

In [ ]:
searchTerm = "hope"
#to apply to one text
with open(Path(sotudir,"2009_Obama.txt"),encoding='utf-8') as f:
    sotu1 = f.read()
tokens = tokenizer.tokenize(sotu1) 
ltokens = [tok.lower() for tok in tokens]
print(getWordFreq(ltokens, searchTerm))

11c. We can then apply this function to the entire SOTU corpus using the tokens list we stored in our dataframe. First, let's review our dataframe:

In [ ]:
sotudf.head()

12. To create a new column in a dataframe, we simply start a line of code with:

```
dfname['newColName'] = [insert instructions for deriving values for column's cells]
```

The following code creates a new column ("wordFreq") which is calculated by applying (.apply() function)) our function "getWordFreq" 
on each value ("x") of the column "ltoks". The getWordFreq reads in not only the value of the ltoks cell (x) but also our searchTerm
(which we set as "government" above)

In [ ]:
searchTerm = "government"
#sotudf['wordFreq'] = sotudf['ltoks'].apply(lambda x: getWordFreq(searchTerm,x))
sotudf['wordFreq'] = sotudf['ltoks'].apply(getWordFreq, term = searchTerm)
sotudf.head()

13. Below we will create a new column to calculate the frequency per million words of our searchTerm.

So, for example, if it appears 10 times in a 10,000 word address, it will have a freq_perMillion score of 1000.

In [ ]:
sotudf['freq_perMillion'] = sotudf['wordFreq']/sotudf['numtoks']*1000000
sotudf.head()

In [ ]:
#this code just temporarily sorts our dataset by the freq_perMillion column (in descending order)
#if we wanted to make this sorting permanent, we would have to add "sotudf = " before the line of code below
sotudf.sort_values('freq_perMillion', ascending = False).head()

14. Now, we will create a simple barplot using the Seaborn package/library for which we assigned the initials "sns" when we imported it at the beginning of this lesson.

Notice how simple this code is. We just identify the dataframe we are drawing data from, and the names of the columns for the x and y data.

In [ ]:
sns.barplot(data = sotudf, x = "year", y = "wordFreq")

15. The previous plot looks similar to the plot we made in the Strings and Files lesson which compared the length of each SOTU address. So, it seems not to be revealing the pattern we want it to reveal.

It is always a good idea in data science to consider the denominator. When should you use absolute values? When should you use percentages or proportions? And, for the latter, what is the correct denominator to choose? In this case, we will use our "freq_perMillion" column which is calculated using the total word (or token) count as our denominator.

In [ ]:
g = sns.barplot(data = sotudf, x = "year", y = "freq_perMillion")
g.tick_params(labelrotation = 90)

<div class="alert alert-info" role="alert"><h3 style = "color:blue">Exercise for Part V.</h1>

<p style = "color:blue">16. Copy and paste the code above in the cell below, but this time use "pres" rather than "year" as our x value.</p>

</div>

## Part VI: Group and Plot Data by President

17. It would help to simplify this visualization. Following *The Atlantic*'s graphic, we can aggregate these results by President. Normally, we would just apply the **groupby** function to group this data by the president's name.

Unfortunately, when I first tried this, I realized that presidents with the same last name were being grouped together (think of the Adamses, Roosevelts, and Bushes). So, first we need to identify each unique president. To do this, we can use the **shift()** function to identify each time a new president's name appears in our chronologically-ordered dataframe. Thus, George H.W. Bush's administration (1989-1993) can be distinguished from his son's (2001-2009). *This works because, fortunately for our purposes, there has always been a gap between two presidents sharing the same name. If <s>Hunter</s>Ashley Biden is our next President then I will need to add first names to our dataset.*

In [ ]:
sotudf["presnum"] = (sotudf["pres"] != sotudf["pres"].shift()).cumsum()
sotudf.head(20)

To explain the code above (sotudf["pres"] != sotudf['pres'].shift()) returns True only when the president in the previous row is different than that in the current row. The function .cumsum() then adds 1 to its running count each time the president changes (indicated by the True value returned by the previous section of code)

You can see the result in the 'presnum' column.

17b. Now, we are going to create a new dataframe ("sotudf2") by grouping together the data in "sotudf" by the "presnum" column (thus aggregating the data by President rather than just by year). The .agg() function then establishes which other columns we want to keep and how the data in those columns will be aggregated ("sum", "mean", "first" being common options). We will then have to re-calculate our proportional variable ("freq_PerMillion").

In [ ]:
sotudf2 = sotudf.groupby(['presnum']).agg({'pres':'first','wordFreq':'sum','numtoks':'sum','year':'first'})
sotudf2['freq_perMillion'] = sotudf2['wordFreq'] / sotudf2['numtoks'] * 1000000
sotudf2.head(10)


18. Okay, let's see what our graphic looks like:

Note: we have cleaned up the axis labels and added a title using the .set command in Seaborn

In [ ]:
g=sns.barplot(data = sotudf2, x = "year", y = "freq_perMillion")
g.tick_params(labelrotation = 90)
g.set(title = f"Frequency of {searchTerm} in State of the Union Addresses")
g.set(ylabel = 'per million words', xlabel = 'President')
g.set(xticklabels = sotudf2.pres); #adding the ";" removes the annoying text that Python sometimes prints out with a graphic#

19. Below, I copied the code from **Step 14 - Step 23**, but this time placed into a function. *Note: all code within a function must be indented. Once the indentation ends, so does the function.* 

In [ ]:
def sotuWordSearch2(ltoksCol,searchTerm): #returns a column of frequencies after searching for a term across a column of lower-case tokens
    #searchTerm = searchTerm.lower()
    wordFreq = ltoksCol.apply(lambda x:collections.Counter(x)[searchTerm])
    return(wordFreq)

def df_wordFreqCalc(old_df,searchTerm):   #reads in a dataframe of SOTU addresses by year and a searchTerm 
                                          ## returns a dataframe aggregated by President, with the 'wordFreq' and 'freq_perMillion' calculated for each president
                                          ## which is calculated using the sotuWordSearch2 function
    requiredCols = ['ltoks','numtoks','pres','presnum','year']
    if not set(requiredCols).issubset(old_df.columns):
        print("missing required column from:",requiredCols)
        return None
    old_df['wordFreq'] = sotuWordSearch2(old_df['ltoks'],searchTerm) 
    new_df = old_df.groupby(['pres','presnum']).agg({'wordFreq':'sum','numtoks':'sum','year':'first'})
    print(new_df.head(2))
    new_df['freq_perMillion'] = new_df['wordFreq'] / new_df['numtoks'] * 1000000
    new_df = new_df.sort_values(['year'])
    new_df = new_df.reset_index()
    print("searching for... :",searchTerm)
    return(new_df)
 
def createWordFreqPlot(df,searchTerm):            #reads in our aggregated SOTU dataframe and creates a bar plot of the search term
    #newdf = df_wordFreqCalc(df,searchTerm)
    requiredCols = ['freq_perMillion','pres','presnum']
    if not set(requiredCols).issubset(df.columns):
        print("missing required column from:",requiredCols)
        return None
    g=sns.barplot(data=df, x="presnum",y="freq_perMillion")
    g.tick_params(labelrotation=90)
    g.set(title = "Frequency of '%s' in State of the Union Addresses"%searchTerm)
    g.set(ylabel='per million words', xlabel='President')
    g.set(xticklabels = df.pres)
    return(g)

20. Now we can use the functions above to quickly choose a new search term, return a dataframe with the results of this search, and then create a plot.

In [ ]:
searchTerm = "freedom"
sotudf3 = df_wordFreqCalc(sotudf,searchTerm)
sotudf3.head(15)

21. Finally we can call our custom plotting function to visualize the results:

In [ ]:
createWordFreqPlot(sotudf3, searchTerm)

22. Experiment by running the code below, but choosing different search terms each time.

In [ ]:
searchTerm = "freedom"
sotudf3 = df_wordFreqCalc(sotudf,searchTerm)
createWordFreqPlot(sotudf3,searchTerm)

23. What additions or changes would make this plot more useful, informative, or eye-catching?

<div class="alert alert-info" role="alert"><h3 style='color:blue'>Exercise (Part VI)</h3>
    <p>23. Explore the dataset by creating bar plots showing the frequency of other words of your choosing. (Hint: we made this really simple with the functions created in Step 24. Just re-use and modify the code we used in Step 25 to call these functions).

</div>